In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import PIL
import pathlib
import tensorflow as tf

In [2]:
data_dir = "/content/drive/MyDrive/main"
data_dir = pathlib.Path(data_dir)
img_height = 180
img_width = 180
batch_size = 32

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed=5,
    class_mode='categorical',
    subset='training') # set as training data

validation_gen = train_datagen.flow_from_directory(
    data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed=5,
    class_mode='categorical',
    subset='validation') 

Found 14141 images belonging to 7 classes.
Found 3535 images belonging to 7 classes.


In [4]:
from keras.applications import  VGG16
VGG_16 = VGG16(include_top=False, 
                     weights=None,
                     input_shape=(img_height, img_width, 3))

In [5]:
VGG_16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 180, 180, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 180, 180, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 90, 90, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 90, 90, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 90, 90, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 45, 45, 128)       0     

In [13]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(VGG_16)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 5, 5, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               3277056   
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 1799      
Total params: 17,993,543
Trainable params: 17,993,543
Non-trainable params: 0
_________________________________________________________________


In [15]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('val_acc')>0.94):
        print("\n 94% val acc reached")
        self.model.stop_training = True

In [ ]:
history = model.fit(train_gen,
                    epochs=25,
                    batch_size=batch_size,
                      callbacks=[CustomCallbacks()],
                    validation_data=validation_gen)

Epoch 1/25
442/442 [==============================] - 8769s 20s/step - loss: 438.7425 - acc: 0.1430 - val_loss: 1.9460 - val_acc: 0.1426
Epoch 2/25
442/442 [==============================] - 175s 396ms/step - loss: 1.9891 - acc: 0.1419 - val_loss: 1.9459 - val_acc: 0.1431
Epoch 3/25
233/442 [==============>...............] - ETA: 1:08 - loss: 1.9459 - acc: 0.1543